In [ ]:
import pandas as pd

test = pd.read_csv('test.csv')
autoru = pd.read_csv('autoru_cleared.csv')
%matplotlib inline

In [ ]:
color_dict = {
    'чёрный': 'black',
    'белый': 'white',
    'серый': 'gray',
    'серебристый': 'silver',
    'синий': 'blue',
    'коричневый': 'gray',
    'красный': 'red',
    'зелёный': 'green',
    'бежевый': 'brown',
    'голубой': 'blue',
    'пурпурный': 'red',
    'золотистый': 'other',
    'фиолетовый': 'violet',
    'жёлтый': 'yellow',
    'оранжевый': 'orange',
    'розовый': 'other',
    'other': 'other'
}
fuel_dict = {
    'gasoline': 'бензин',
    'diesel': 'дизель',
    'gas': 'газ',
    'hybrid-petrol': 'гибрид',
    'electric': 'электро',
    'hybrid-diesel': 'гибрид'
}
body_dict = {
    'внедорожник 5 дв.': 'suv',
    'седан': 'sedan',
    'лифтбек': 'liftback',
    'хэтчбек 5 дв.': 'hatchback',
    'универсал 5 дв.': 'universal',
    'универсал 3 дв.': 'universal',
    'минивэн': 'minivan',
    'купе': 'coupe',
    'компактвэн': 'minivan',
    'хэтчбек 3 дв.': 'hatchback',
    'пикап двойная кабина': 'pickup',
    'внедорожник 3 дв.': 'suv',
    'купе-хардтоп': 'coupe',
    'кабриолет': 'cabriolet',
    'фургон': 'van',
    'родстер': 'coupe',
    'микровэн': 'minivan',
    'седан-хардтоп': 'sedan',
    'пикап одинарная кабина': 'pickup',
    'пикап полуторная кабина': 'pickup',
    'внедорожник открытый': 'suv',
    'лимузин': 'limousine',
    'тарга': 'cabriolet',
    'седан 2 дв.': 'coupe',
    'фастбек': 'hatchback',
    'хэтчбек 4 дв.': 'hatchback',
    'фаэтон': 'cabriolet'
    
}
transmission_dict = {
    'автоматическая': 'automatic',
    'автомат': 'automatic',
    'механическая': 'mechanical',
    'механика': 'mechanical',
    'роботизированная': 'robot',
    'робот': 'robot',
    'вариатор': 'variator'
}
drivetrain_dict = {
    'полный': 'all',
    'передний': 'front',
    'задний': 'rear'
}

In [ ]:
def clear_eng_power(hp):
    capacity = str(hp).split()[0]
    if capacity == 'undefined':
        return 2.0
    return float(capacity)

def eval_features(feature_string):
    feature_count = 0
    if len(feature_string) > 5:
        for i in eval(feature_string[2:-2]):
            feature_count += len(i['values'])
        return feature_count
    else:
        return 0

test['color'] = test['color'].apply(lambda x:color_dict[x])
#test['bodyType'] = test['bodyType'].apply(lambda x:body_dict[x])
test['vehicleTransmission'] = test['vehicleTransmission'].apply(lambda x:transmission_dict[x])
test['Привод'] = test['Привод'].apply(lambda x:drivetrain_dict[x])
test['productionDate'] = test['productionDate'].apply(lambda x:int(x))
test['mileage'] = test['mileage'].apply(lambda x:int(x))
test['enginePower'] = test['enginePower'].apply(clear_eng_power)
test['engineDisplacement'] = test['engineDisplacement'].apply(clear_eng_power)
test['features_count'] = (test['Комплектация'].apply(eval_features)//8).apply(lambda x: x if x<10 else 9)

test = test.rename(columns={'bodyType': 'body_type',
                            'brand': 'manufacturer_name',
                            'mileage': 'odometer_value',
                            'vehicleTransmission': 'transmission',
                            'productionDate': 'year_produced',
                            'fuelType': 'engine_fuel',
                            'engineDisplacement': 'engine_capacity',
                            'enginePower': 'engine_power',
                            'Привод': 'drivetrain'
})

test['year_produced'] = test['year_produced'] - 1938

In [ ]:
autoru = autoru[autoru['manufacturer_name'].isin(test['manufacturer_name'])]

autoru['power_per_volume'] = autoru['engine_power']/autoru['engine_capacity']
test['power_per_volume'] = test['engine_power']/test['engine_capacity']

country_dict = {
'TOYOTA': 'JP',
'VOLKSWAGEN': 'DE',
'MERCEDES': 'DE',
'BMW': 'DE',
'NISSAN': 'JP',
'SKODA': 'CH',
'AUDI': 'DE',
'MITSUBISHI': 'JP',
'HONDA': 'JP',
'VOLVO': 'SW',
'LEXUS': 'JP',
'SUZUKI': 'JP',
'INFINITI': 'JP'
}

autoru['manufacturer_country'] = autoru['manufacturer_name'].replace(to_replace=country_dict)
test['manufacturer_country'] = test['manufacturer_name'].replace(to_replace=country_dict)

autoru['odometer_per_year'] = (autoru['odometer_value']+1) / (83 - autoru['year_produced'])
test['odometer_per_year'] = (test['odometer_value']+1) / (83 - test['year_produced'])

autoru = autoru[autoru['engine_capacity'] < 7]

autoru = autoru[autoru['odometer_value'] > 0]

In [ ]:
def calculate_tax(power):
    tax_dict ={
        '100': 2.5,
        '150': 3.5,
        '200': 5,
        '250': 7.5,
        'max': 15
    }
    key = ''
    
    if power <= 100:
        key = '100'
    elif power <= 150:
        key = '150'
    elif power <= 200:
        key = '200'
    elif power <= 250:
        key = '250'
    else:
        key = 'max'
    
    return power*tax_dict[key]
                    
        
autoru['tax'] = autoru['engine_power'].apply(calculate_tax)
test['tax'] = test['engine_power'].apply(calculate_tax)

In [ ]:
import numpy as np

features = ['body_type',#0
            'manufacturer_name',#1
            'transmission',#2
            'year_produced',#3
            'engine_fuel',#4
            'color',#5
            'drivetrain',#6
            'engine_capacity',#7
            'engine_power',#8
            'odometer_value',#9
            'power_per_volume',#10
            'manufacturer_country',#11
            'odometer_per_year',#12
            #'tax'#13
]


autoru = autoru[features+['price']]
test = test[features]

In [ ]:
N_ESTIMATORS = 250

#Random Forest prediction
from sklearn.ensemble import RandomForestRegressor


autoru_rfr = pd.concat([autoru, pd.get_dummies(autoru[['transmission', 'drivetrain', 'engine_fuel']])], axis=1)
test_rfr = pd.concat([test, pd.get_dummies(test[['transmission', 'drivetrain', 'engine_fuel']])], axis=1)

autoru_rfr['engine_fuel_электро'] = 0

rfr_drop = ['body_type', 'manufacturer_name', 'transmission',
           'engine_fuel', 'color', 'drivetrain','manufacturer_country',]

autoru_rfr.drop(rfr_drop, axis=1, inplace=True)
test_rfr.drop(rfr_drop, axis=1, inplace=True)

model_rfr = RandomForestRegressor(n_estimators=N_ESTIMATORS, n_jobs=-1, verbose=1)
model_rfr.fit(autoru_rfr.drop(['price'], axis=1), autoru_rfr['price'].apply(np.log1p))

test['rfr_prediction'] = model_rfr.predict(test_rfr)
autoru['rfr_prediction'] = model_rfr.predict(autoru_rfr.drop(['price'], axis=1))

In [ ]:
from sklearn.model_selection import train_test_split

X = autoru.drop(['price'], axis=1)
y = autoru['price'].apply(np.log1p)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=42)

In [ ]:
ITERATIONS = 1000
LR         = 0.2

from catboost import CatBoostRegressor
model = CatBoostRegressor(iterations = ITERATIONS,
                          learning_rate = LR,
                          #random_seed = 42,
                          #depth = 6,
                          #l2_leaf_reg = 1,
                          eval_metric = 'MAPE',
                          custom_metric = ['R2', 'MAE'],
                          early_stopping_rounds = 500
                         )

model.fit(X_train, y_train,
            cat_features=[0,1,2,4,5,6,11],
            eval_set=(X_test, y_test),
            verbose_eval=500,
            use_best_model=True,
            plot=True
            )

In [ ]:
model.get_feature_importance(prettified=True) #0.00278

In [ ]:
pred = model.predict(test)
sample_submission = pd.DataFrame()
sample_submission['price'] = pred
sample_submission['price'] = sample_submission['price'].apply(np.expm1) #REVERT LOG1P TRANSFORMATION
#sample_submission['price'] = sample_submission['price']*0.95
sample_submission = sample_submission.reset_index().rename(columns={'index': 'id'})
sample_submission.to_csv(f'submission_v09.csv', index=False)
sample_submission.head()